#### DB 초기화 프로그램

In [3]:
import pymysql
import json

with open('../mysql.json') as fp:
    config_str = fp.read()
config = json.loads(config_str)

config

{'host': '140.238.202.82',
 'user': 'mcuser',
 'password': 'Gratis2021!',
 'database': 'mcdb',
 'port': 3306}

In [4]:
conn = pymysql.connect(host=config['host'], user=config['user'], password=config['password'],
                       database=config['database'], port=config['port'])

#### new_products 테이블

In [8]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists new_products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [9]:
cur = conn.cursor()
cur.execute(sql)

0

In [14]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품','아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV','카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리','훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [16]:
[len(x) for x in [camping_goods, car_goods, health_goods, category_names]]

[10, 10, 10, 3]

In [17]:
import random
random.seed(2021)

for item in camping_goods:
    price = random.randint(100, 999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, "캠핑", cost)

텐트 95600 캠핑 87000
타프 97800 캠핑 92910
천막 65700 캠핑 58470
침낭 35300 캠핑 33540
매트 13500 캠핑 12420
랜턴 58500 캠핑 54990
취사용품 16500 캠핑 14850
아이스박스 37600 캠핑 33460
보조배터리 58600 캠핑 50400
워터저그 26500 캠핑 23060


In [18]:
sql = "insert into new_products(pname, pprice, pcategory, pcost) values(%s, %s, %s, %s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100, 999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item, price, category_names[i], cost))
conn.commit()

#### new_sales 테이블

In [1]:
# spid를  외래키(foreign key)로 설정
sql = """
    CREATE TABLE if NOT exists new_sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES new_products(pid)
    ) AUTO_INCREMENT=101;
"""

In [5]:
cur = conn.cursor()
cur.execute(sql)

0

In [10]:
calendar = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
company_list = ['멀티캠퍼스', '한경아카데미', '표준협회', '생산성본부', '테크브루']

for i in range(10):
    date = f'2022-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11, 40)
    unit = random.randint(1, 20)
    print(date, company, pid, unit)

2022-1-18 테크브루 27 16
2022-1-4 생산성본부 12 14
2022-1-17 한경아카데미 16 19
2022-1-2 한경아카데미 38 15
2022-1-3 테크브루 18 13
2022-1-16 표준협회 24 12
2022-1-31 테크브루 27 17
2022-1-6 테크브루 29 5
2022-1-10 표준협회 38 18
2022-1-14 표준협회 22 10


In [11]:
sql = "INSERT INTO new_sales VALUES(default, %s, %s, %s, %s);"

for month, day in calendar.items():
    for i in range(10):
        date = f'2022-{month}-{random.randint(1, day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11, 40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

#### 두 테이블을 Join한 View 생성

In [12]:
sql = """
    CREATE VIEW new_sales_book as 
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM new_sales
        JOIN new_products
        ON new_sales.spid = new_products.pid
        ORDER BY sdate;
"""

In [13]:
cur = conn.cursor()
cur.execute(sql)

0

#### 데이터를 sales_book.csv로 저장

In [14]:
sql = "SELECT * FROM new_sales_book"
cur.execute(sql)
results = cur.fetchall()

In [15]:
import pandas as pd

df = pd.DataFrame(data=results)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       120 non-null    int64 
 1   1       120 non-null    object
 2   2       120 non-null    object
 3   3       120 non-null    int64 
 4   4       120 non-null    object
 5   5       120 non-null    object
 6   6       120 non-null    int64 
 7   7       120 non-null    int64 
 8   8       120 non-null    int64 
 9   9       120 non-null    int64 
 10  10      120 non-null    int64 
dtypes: int64(7), object(4)
memory usage: 10.4+ KB


In [16]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,104,2022-01-05,멀티캠퍼스,22,내비게이션,자동차용품,44500,18,42280,801000,39960
1,101,2022-01-07,한경아카데미,19,보조배터리,캠핑용품,75000,8,69000,600000,48000
2,109,2022-01-08,생산성본부,17,취사용품,캠핑용품,57800,18,49710,1040400,145620
3,102,2022-01-15,멀티캠퍼스,37,트램펄린,헬스용품,93000,2,81840,186000,22320
4,107,2022-01-17,멀티캠퍼스,28,헤드유닛,자동차용품,21800,19,20710,414200,20710


In [17]:
del df[3]
del df[0]

In [18]:
df.columns = ['판매일자', '판매처', '상품명', '카테고리', '단가', '수량', '원가', '매출', '이익']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판매일자    120 non-null    object
 1   판매처     120 non-null    object
 2   상품명     120 non-null    object
 3   카테고리    120 non-null    object
 4   단가      120 non-null    int64 
 5   수량      120 non-null    int64 
 6   원가      120 non-null    int64 
 7   매출      120 non-null    int64 
 8   이익      120 non-null    int64 
dtypes: int64(5), object(4)
memory usage: 8.6+ KB


In [19]:
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출,이익
0,2022-01-05,멀티캠퍼스,내비게이션,자동차용품,44500,18,42280,801000,39960
1,2022-01-07,한경아카데미,보조배터리,캠핑용품,75000,8,69000,600000,48000
2,2022-01-08,생산성본부,취사용품,캠핑용품,57800,18,49710,1040400,145620
3,2022-01-15,멀티캠퍼스,트램펄린,헬스용품,93000,2,81840,186000,22320
4,2022-01-17,멀티캠퍼스,헤드유닛,자동차용품,21800,19,20710,414200,20710


In [20]:
df.to_csv('sales_book.csv', index=False)

#### 마무리

In [21]:
cur.close()
conn.close()